In [1]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

def connect_to_cassandra():
    cluster = Cluster(['localhost'])  # Replace with your Cassandra node IP
    session = cluster.connect('spark_streams')  # Replace with your keyspace name
    return session

In [31]:
def query_request_count_by_project(session, project_id, from_date):
    query = f"""
        SELECT * FROM aggregated_api_request
        WHERE project_id = '{project_id}' AND timestamp >= '{from_date}';
    """
    statement = SimpleStatement(query)
    result = session.execute(statement)

    return result

In [91]:
session = connect_to_cassandra()
project_id = 'project-id-1'  # Replace with the desired project ID# Replace with the desired month bucket
start = "2023-11-01 00:00:00.000"

result = query_request_count_by_project(session, project_id, start)
df = pd.DataFrame(result)

df_resampled =  df.set_index("timestamp").resample("10Min").sum().fillna(0).reset_index()
project_id = "project-id-1"

df_success = df[df['event_type'] == 'PREDICT_COMPLETED']
df_failed = df[df['event_type'] == 'PREDICT_FAILED']

df_resampled['project_id'] = [project_id if project == 0 else project for project in df_resampled['project_id']]
df_failed

,project_id,timestamp,avg_duration,count,event_type,fiftieth_per_duration,ninetieth_per_duration,ninety_fifth_per_duration
0,project-id-1,2023-11-30 07:10:00,7222.222168,9,PREDICT_FAILED,6000.0,14000.0,14000.0
1,project-id-1,2023-11-30 07:20:00,7902.173828,92,PREDICT_FAILED,8000.0,14000.0,14000.0
5,project-id-1,2023-12-01 07:00:00,7111.111328,18,PREDICT_FAILED,6000.0,12000.0,15000.0
8,project-id-1,2023-12-01 07:40:00,9000.000000,1,PREDICT_FAILED,9000.0,9000.0,9000.0
9,project-id-1,2023-12-01 07:50:00,7185.185059,27,PREDICT_FAILED,7000.0,15000.0,15000.0
14,project-id-1,2023-12-02 06:30:00,8210.526367,38,PREDICT_FAILED,8000.0,14000.0,15000.0


In [8]:
import pytz

def process_dataframe_for_last(df, time_range='6_hours'):
    
    end_time = pd.to_datetime('now')
    timedelta = 10
    # Determine the start time based on the specified time range
    if time_range == '6_hours':
        end_time = end_time.replace(second=0, microsecond=0, minute=0, hour=end_time.hour+1)
        start_time = (end_time - pd.Timedelta(hours=6))
    elif time_range == '1_day':
        end_time = pd.to_datetime(end_time.date()) + pd.DateOffset(days=1)
        start_time = (end_time - pd.Timedelta(days=1)).date()
        timedelta = 60
    elif time_range == '7_day':
        end_time = pd.to_datetime(end_time.date()) + pd.DateOffset(days=1) - pd.Timedelta(seconds=1)
        timedelta = 60 * 24
        start_time = pd.to_datetime((end_time - pd.Timedelta(days=8)).date())
    else:
        raise ValueError("Invalid time_range. Supported values are '6_hours' and 'one_day'.")

    desired_timezone = 'Asia/Bangkok'
    df['timestamp'] = pd.to_datetime('now').tz_localize(pytz.timezone(desired_timezone)).strftime('%Y-%m-%d %H:%M:%S')
    # Filter data for the specified time range
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df_filtered = df[df['timestamp'] >= start_time]

     # Convert 'timestamp' to DateTime object

    if time_range == '6_hours':
        df_filtered['formatted_timestamp'] = df_filtered['timestamp'].dt.round('10min')
    elif time_range == '1_day':
        df_filtered['formatted_timestamp'] = df_filtered['timestamp'].dt.floor('H')
    elif time_range == '7_day':
        df_filtered['formatted_timestamp'] = df_filtered['timestamp'].dt.floor('D')

    # Group by formatted timestamp and calculate count
    grouped_df = df_filtered.groupby(['formatted_timestamp'])['count'].sum().reset_index()

    chart_data = {
        'timestamp': [],
        'count': []
    }

    def formatTimestime(timestamp): 
        if time_range == '6_hours':
            return timestamp.strftime('%H:%M')
        elif time_range == '1_day':
            return timestamp.strftime('%H')
        else:
            return timestamp.strftime('%Y-%m-%d')

    current_time = start_time
    while current_time <= end_time:
        count_value = grouped_df.loc[grouped_df['formatted_timestamp'] == current_time, 'count'].values
        chart_data['count'].append(count_value[0] if len(count_value) > 0 else 0)
        chart_data['timestamp'].append(formatTimestime(current_time))
        current_time += pd.Timedelta(minutes=timedelta)     
    
     
    return df_filtered

In [9]:
res = process_dataframe_for_last(df_resampled, '6_hours')
res

,timestamp,project_id,avg_duration,count,event_type,fiftieth_per_duration,ninetieth_per_duration,ninety_fifth_per_duration,formatted_timestamp
0,2023-12-02 12:24:20,project-id-1,7222.222168,9,PREDICT_FAILED,6000.0,14000.0,14000.0,2023-12-02 12:20:00
1,2023-12-02 12:24:20,project-id-1,7902.173828,92,PREDICT_FAILED,8000.0,14000.0,14000.0,2023-12-02 12:20:00
2,2023-12-02 12:24:20,project-id-1,7721.311523,183,PREDICT_COMPLETED,7000.0,14000.0,15000.0,2023-12-02 12:20:00
3,2023-12-02 12:24:20,project-id-1,8430.051758,193,PREDICT_COMPLETED,8000.0,15000.0,15000.0,2023-12-02 12:20:00
4,2023-12-02 12:24:20,project-id-1,7560.000000,25,PREDICT_COMPLETED,8000.0,15000.0,15000.0,2023-12-02 12:20:00
...,...,...,...,...,...,...,...,...,...
273,2023-12-02 12:24:20,project-id-1,0.000000,0,0,0.0,0.0,0.0,2023-12-02 12:20:00
274,2023-12-02 12:24:20,project-id-1,0.000000,0,0,0.0,0.0,0.0,2023-12-02 12:20:00
275,2023-12-02 12:24:20,project-id-1,0.000000,0,0,0.0,0.0,0.0,2023-12-02 12:20:00
276,2023-12-02 12:24:20,project-id-1,9500.000000,8,PREDICT_COMPLETED,8000.0,15000.0,15000.0,2023-12-02 12:20:00


In [92]:
def process_dataframe_for_last_6hour(df, time_range='6_hours'):
    # Get current date and time
    now = datetime.now()
    now = pd.to_datetime(now)
    rounded_now = now.round('10min')

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC')
    df['timestamp'] = pd.to_datetime(pd.to_datetime(df['timestamp']).dt.tz_convert('Asia/Bangkok')).dt.strftime('%Y-%m-%d %H:%M:%S')
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    end = rounded_now
    start = end - pd.Timedelta(hours=6)
    if time_range == "7_days":
        end = pd.to_datetime(now.date())
        start = end - pd.Timedelta(days=8)

    df_filtered = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]

    if time_range == '6_hours':
        df_filtered['timestamp_formated']= df_filtered['timestamp'].dt.strftime('%H:%M')
    else:
        df_filtered['timestamp_formated']= df_filtered['timestamp'].dt.strftime('%Y-%m-%d')

    # df_filtered['timestamp'] = df_filtered['timestamp'].apply(formatTimestime)

    return df_filtered


session = connect_to_cassandra()
project_id = 'project-id-1'  # Replace with the desired project ID# Replace with the desired month bucket
start = "2023-11-01 00:00:00.000"

result = query_request_count_by_project(session, project_id, start)
df = pd.DataFrame(result)

df_resampled =  df.set_index("timestamp").resample("10Min").sum().fillna(0).reset_index()
project_id = "project-id-1"

df_resampled['project_id'] = [project_id if project == 0 else project for project in df_resampled['project_id']]
df_resampled
res = process_dataframe_for_last_6hour(df_failed)
res

/var/folders/sx/213nq68x0m575_s41z6cc3ch0000gn/T/ipykernel_58363/4093044004.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['timestamp_formated']= df_filtered['timestamp'].dt.strftime('%H:%M')


,project_id,timestamp,avg_duration,count,event_type,fiftieth_per_duration,ninetieth_per_duration,ninety_fifth_per_duration,timestamp_formated
14,project-id-1,2023-12-02 13:30:00,8210.526367,38,PREDICT_FAILED,8000.0,14000.0,15000.0,13:30


In [94]:
from datetime import timedelta


def round_up_to_10_minutes(dt):
    remainder = dt.minute % 10
    if remainder == 0:
        return dt
    else:
        return dt + timedelta(minutes=(10 - remainder))


# Get current date and time
now = datetime.now()
now = pd.to_datetime(now)
rounded_now = round_up_to_10_minutes(now)
rounded_now


Timestamp('2023-12-02 13:40:07.404656')